In [14]:
import torch 
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

tf =transforms.Compose( [
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

train_dl = DataLoader(
    datasets.ImageFolder("data/Training", tf),
    batch_size=32, shuffle=True, num_workers=4, pin_memory=True
)

test_dl = DataLoader(
    datasets.ImageFolder("data/Validation", tf),
    batch_size=32, shuffle=False, num_workers=4, pin_memory=True
)

Using device: cpu


In [15]:
model = nn.Sequential(
    nn.Conv2d(3, 32, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(128 * 16 * 16, 256), nn.ReLU(), nn.Dropout(0.5),
    nn.Linear(256, 4) 
).to(device)


In [16]:
opt = optim.Adam(model.parameters(), 1e-4)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
model.train()

for epoch in range(10):
    running_loss = 0.0

    for x, y in train_dl:
        x, y = x.to(device), y.to(device)  

        opt.zero_grad()

        outputs = model(x)
        loss = loss_fn(outputs, y)
        loss.backward()
        opt.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_dl)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")


Epoch 1, Loss: 0.5948
Epoch 2, Loss: 0.4911
Epoch 3, Loss: 0.4435
Epoch 4, Loss: 0.3806
Epoch 5, Loss: 0.3468
Epoch 6, Loss: 0.3139
Epoch 7, Loss: 0.2776
Epoch 8, Loss: 0.2612
Epoch 9, Loss: 0.2338
Epoch 10, Loss: 0.2104


In [19]:
model.eval
test_loss, correct = 0.0, 0

with torch.no_grad():
    for x, y in test_dl:
        x, y = x.to(device), y.to(device)

        logits = model(x)
        test_loss += loss_fn(logits, y).item() * x.size(0)

        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()

test_loss /= len(test_dl.dataset)
accuracy = 100.0 * correct / len(test_dl.dataset)


print(f"Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%")

d:\AI Projects\Brain Tumor Classifier\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Test Loss: 0.1900, Accuracy: 92.66%


In [ ]:
import random
import matplotlib.pyplot as plt
from torchvision.transforms.functional import to_pil_image
import torch

model.eval()

idx = random.randrange(len(test_dl.dataset))
img, label = test_dl.dataset[idx]


unnorm = img * 0.5 + 0.5
plt.imshow(to_pil_image(unnorm))
plt.axis('off')
plt.title('Sample from test set')
plt.show()

with torch.no_grad():
    logits = model(img.unsqueeze(0).to(device))
    pred = logits.argmax(1).item()

class_names = test_dl.dataset.classes
print(f"Predicted class: {class_names[pred]}")
print(f"Ground-truth: {class_names[label]}")